In [1]:
# exif2csv - Read exif lat/lon/alt and convert to NAD83 / NAVD88
# csherwood@usgs.gov

import __future__
import sys
# argument parsing
import argparse
# glob for files
import glob
# file path tools
import os
# os.path import join, basename, splitdrive, split
# regular expression
import re
# math library
import math
# Numpy
import numpy as np

# GDAL Python library
from osgeo import osr
# EXIF Reader
import exifread
# Image library
from PIL import Image

import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')

Disclaimer

Unless otherwise noted, the software and content on this site is
in the public domain because it contains materials developed by
the United States Geological Survey, an agency of the United States
Department of Interior. For more information, see the official USGS
copyright policy at:

http://www.usgs.gov/visual-id/credit_usgs.html#copyright

This software and content is distributed on an "AS IS" BASIS, WITHOUT
WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. Any
dependent libraries included here are distributed under open source
(or open source-like) licenses/agreements. Appropriate license agreements
are included with each library.


In [2]:
src_crs = 'EPSG:4326' 
dst_crs = 'EPSG:3724' 
#dst_crs = 'EPSG:5498'
#EPSG:3724 is NAD83(NRS 2007)
#EPSG:5498 is "compound" NAD83+NAVD88; does not do what I hoped
#EPSG:5038 is NAVD88
#
# source coordinate system
srs_cs = osr.SpatialReference()
srs_cs.SetFromUserInput(src_crs)

# destination coordinate system
dst_cs = osr.SpatialReference()
dst_cs.SetFromUserInput(dst_crs)

# osr image transformation object
transform = osr.CoordinateTransformation(srs_cs, dst_cs)

# print coordinate system information
print " >> SOURCE COORDINATE REFERENCE SYSTEM: "
print srs_cs

print " >> DESTINATION COORDINATE REFERENCE SYSTEM: "
print dst_cs

 >> SOURCE COORDINATE REFERENCE SYSTEM: 
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]
 >> DESTINATION COORDINATE REFERENCE SYSTEM: 
PROJCS["NAD83(NSRS2007) / UTM zone 17N",
    GEOGCS["NAD83(NSRS2007)",
        DATUM["NAD83_National_Spatial_Reference_System_2007",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6759"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4759"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["cen

In [3]:
# test with one of Maitane's target locations
lat = 29.68366631
lon = -81.21871357
alt = -27.7583
# looking for: 478838.9754, 3283753.8587, 1.0327,
easting,northing,elev = transform.TransformPoint(lon, lat, alt)
print easting, northing, elev

478838.975621 3283753.8589 -27.7583


In [4]:
# file for image locations
deltaz = 41. # approximate offset from altitude to NAVD88

fnout = "D:\\2017_Matthew\\maitane\\2016-10-30_images_UTM.csv"
header_out = "Image, Lat (WGS84), Lon (WGS84), Alt. (m), Easting (m NAD83 UTM17N), Northing (m NAD83 UTM17N), Elev. (m NADV88)"
fo = open(fnout, 'w')
print header_out
fo.write(header_out+"\n")

# path to images
filepath = "D:\\2017_Matthew\\maitane\\2016-10-30_low_tide\\*.jpg"
files_gps = []
files = sorted(glob.glob(filepath))
n_images = len(files)
print("Found {0} files.\n".format(n_images))

n_images_GPS = 0
for file in files:
    fs = open(file,'rb')

    # attempt to read files
    try:
        tags = exifread.process_file(fs)
    except:
        print "No EXIF Data"
    try:
        try:
            lonstr = tags['GPS GPSLongitude']
            latstr = tags['GPS GPSLatitude']
        except:
            print 'failed latlon'
        try:
            latref = tags['GPS GPSLatitudeRef']
            lonref = tags['GPS GPSLongitudeRef']
        except:
            print 'failed latref or lonref'
                                        
        files_gps.append(file)
        n_images_GPS += 1
    except KeyError:
        print "No GPS data for this image: ",file

    try:
        altstr = tags['GPS GPSAltitude']
    except:
        print "No altitude data for this image: ",file
    try:
        altref = tags['GPS GPSAltitudeRef']
    except:
        print "No altitude ref for this image: ",file
    
    ## Latitude
    # use regular expression to get value between brackets
    y_regx = re.search('\[(.*?)\]', str(latstr))
    y_regx = y_regx.group(1)
    y_regx = y_regx.split(', ')
    # properly convert values to floats
    y = []
    y.append(eval(compile(y_regx[0],'<string>','eval', __future__.division.compiler_flag)))
    y.append(eval(compile(y_regx[1],'<string>','eval', __future__.division.compiler_flag)))
    y.append(eval(compile(y_regx[2],'<string>','eval', __future__.division.compiler_flag)))

    ## Longitude
    x_regx = re.search('\[(.*?)\]', str(lonstr))
    x_regx = x_regx.group(1)
    x_regx = x_regx.split(', ')
    x = []
    x.append(eval(compile(x_regx[0],'<string>','eval', __future__.division.compiler_flag)))
    x.append(eval(compile(x_regx[1],'<string>','eval', __future__.division.compiler_flag)))
    x.append(eval(compile(x_regx[2],'<string>','eval', __future__.division.compiler_flag)))
    
    # convert from latitude longitude minutes seconds to decimal
    lon = (np.sign(x[0]))*(math.fabs(x[0]) + (x[1]/60.0) + (x[2]/3600.0))
    lat = (np.sign(y[0]))*(math.fabs(y[0]) + (y[1]/60.0) + (y[2]/3600.0))
    # handle longitude west values to negative
    if str(lonref) == 'W':
        lon = -1.0*lon
    # handle latitude south values to negative
    if str(latref) == 'S':
        lon = -1.0*lat
        
    ## Altitude
    # This is how altitude appears in the EXIF info
    # altstr = "(0x0006) Ratio=918/25 @ 1783"
    # Extract the string following "Ratio=" until a blank space
    print altstr
    #s1,e1 = re.search("Ratio=",altstr).span()
    #print s1,e1
    #s2,e2 = re.search(" ",str[e1:]).span()
    #print s2,e2
    #altrat=altstr[e1:e1+s2]
    #print altrat
    # Turn the string into an expression and compute
    altz=eval(compile(altrat,'<string>','eval', __future__.division.compiler_flag))

    print( file, lat, lon, lonstr, altz )
    #e,n,zt = transform.TransformPoint(lon, lat, alt)
    #znavd88 = zt+deltaz
    #print(file, e, n, znavd88)
    #print("{0:s}, {1:.6f}, {2:.6f}, {3:.4f}, {4:.4f}, {5:.4f}, {6:.4f}".format(file,lat,lon,ze,e,n,znavd88))
    #fo.write("{0:s}, {1:.6f}, {2:.6f}, {3:.4f}, {4:.4f}, {5:.4f}, {6:.4f}\n".format(file,lat,lon,ze,e,n,znavd88))
fo.close


Image, Lat (WGS84), Lon (WGS84), Alt. (m), Easting (m NAD83 UTM17N), Northing (m NAD83 UTM17N), Elev. (m NADV88)
Found 262 files.

918/25


TypeError: expected string or buffer

In [ ]:
    n_images_GPS = 0
    for file in files:
        fs = open(file,'rb')

        # intialize arrays
        lat = []
        lon = []
        latref = []
        lonref = []
        alt = []
        time = []

        # attempt to read files
        try:
            tags = exifread.process_file(fs)
        except:
            print "No EXIF Data"
        try:
            try:
                lon = tags['GPS GPSLongitude']
                lat = tags['GPS GPSLatitude']
            except:
                print 'failed latlon'
            try:
                latref = tags['GPS GPSLatitudeRef']
                lonref = tags['GPS GPSLongitudeRef']
            except:
                print 'failed latref or lonref'